# Earnings Call Transcript Analysis

## Setup

In [1]:
# %%writefile requirements.txt
# llama-index
# llama-index-llms-huggingface
# llama-index-embeddings-fastembed
# fastembed
# Unstructured[md]
# chromadb
# llama-index-vector-stores-chroma
# llama-index-llms-groq
# einops
# accelerate
# sentence-transformers
# llama-index-llms-mistralai
# llama-index-llms-openai

In [8]:
from helper import get_openai_api_key, display_text
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Chat Engine for One Transcript

In [4]:
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex,
    SummaryIndex
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o",temperature=0.1)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [6]:
# Set url and dir
transcripts_dir = "transcripts/NVDA.pdf"
url_1="https://eugeneyan.com/writing/llm-patterns/"
url_2="https://hindenburgresearch.com/smci/"
working_url = url_1

In [9]:
## Simple ChatEngine
from llama_index.core.chat_engine import SimpleChatEngine
chat_engine = SimpleChatEngine.from_defaults()
response = chat_engine.chat("summarize the transcript")
display_text(response)

## Setup an agent

We can create vector search and summary tools respectively for each of the 3 papers, which then result in 6 tools in total. These tools can then be made available to the agent worker. 

In [10]:
import os
root_path = "transcripts"
file_name = []
file_path = []
for file in os.listdir(root_path):
  if file.endswith(".pdf"):
    file_name.append(file.split(".")[0])
    file_path.append(os.path.join(root_path,file))
#
print(file_name)
print(file_path)

['tgt', 'LULU', 'snow', 'meta', 'shop', 'walmart', 'NVDA', 'PDD', 'low']
['transcripts/tgt.pdf', 'transcripts/LULU.pdf', 'transcripts/snow.pdf', 'transcripts/meta.pdf', 'transcripts/shop.pdf', 'transcripts/walmart.pdf', 'transcripts/NVDA.pdf', 'transcripts/PDD.pdf', 'transcripts/low.pdf']


In [104]:
from utils import get_doc_tools
from pathlib import Path

papers_to_tools_dict = {}
for name,filename in zip(file_name,file_path):
  vector_tool,summary_tool = get_doc_tools(filename,name)
  papers_to_tools_dict[name] = [vector_tool,summary_tool]

KeyboardInterrupt: 

In [ ]:
initial_tools = [t for f in file_name for t in papers_to_tools_dict[f]]

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

In [ ]:
len(initial_tools)

10

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

system_message = """ \
You are an agent designed to answer queries over a set of given earnings call transcripts reports.
Please read each document thorougly and always use the tools provided to answer a question. Use the original quote as much as possible. Do not rely on prior knowledge.\
"""

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    system_prompt=system_message,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
# The agent can then choose the appropriate tools for each steps 
response = agent.query(
    "Tell me about how Meta uses AI in their business? please be as detailed as possible."
    "and what results they have seen so far"
    "and if they mention any capex or opex related to AI"
)

Added user message to memory: Tell me about how Meta uses AI in their business? please be as detailed as possible.and what results they have seen so farand if they mention any capex or opex related to AI
=== Calling Function ===
Calling function: vector_tool_meta with args: {"query": "How does Meta use AI in their business?"}
=== Function Output ===
Meta uses AI in their business for various purposes such as content moderation, recommendations of content and ads, video monetization, simplifying the creation of video ads, transforming static images into more dynamic content, and enhancing consumer engagement.
=== Calling Function ===
Calling function: vector_tool_meta with args: {"query": "What results has Meta seen from using AI?"}
=== Function Output ===
Meta has seen positive results from using AI, particularly in content moderation where most of the work is done through AI. The metrics show progress in identifying and taking action on harmful content before it is reported. Additiona

In [ ]:
# we can chat/query about the LLM's reasoning process
response = agent.chat(
    "Please read each document carefully and summarize how each company view the consumer spending."
)

Added user message to memory: Please read each document carefully and summarize how each company view the macro enviroment.
=== Calling Function ===
Calling function: summary_tool_meta with args: {"input": "How does Meta view the macro environment?"}
=== Function Output ===
Meta views the macro environment as challenging due to factors such as economic uncertainty, the impact of the war in Ukraine, rising inflation, potential recession, the impact of COVID-19 on certain sectors, and overall market volatility. Despite these challenges, Meta remains focused on being disciplined in spending while investing in key areas that will drive growth as the economic environment improves. The company believes in its resilience and ability to adapt to changes in consumer behavior and market needs in the long term.
=== Calling Function ===
Calling function: summary_tool_shop with args: {"input": "How does Shopify view the macro environment?"}
=== Function Output ===
Shopify views the macro environmen

In [ ]:
print(response.metadata)

None


## 2. Setup an agent over more files

In [ ]:
from utils import get_doc_tools

paper_to_tools_dict = {}
for name,filename in zip(file_name,file_path):
  print(f"Getting tools for paper: {filename}")
  vector_tool, summary_tool = get_doc_tools(filename,name)
  paper_to_tools_dict[name] = [vector_tool, summary_tool]

Getting tools for paper: transcripts/tgt.pdf
Getting tools for paper: transcripts/meta.pdf
Getting tools for paper: transcripts/shop.pdf
Getting tools for paper: transcripts/walmart.pdf
Getting tools for paper: transcripts/low.pdf


### Extend the Agent with Tool Retrieval

In [ ]:
all_tools = [t for f in file_name for t in papers_to_tools_dict[f]]

In [ ]:
# define an "object" index over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [ ]:
# define the "retriever" over the index with specified retrieval method
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [ ]:
tools = obj_retriever.retrieve(
    "Tell me how company talk about consumer spending in their earnings call."
)

In [ ]:
# check for the top 3 tools selected
tools[0].metadata

ToolMetadata(description='Use ONLY IF you want to get a holistic summary of MetaGPT. Do NOT use if you have specific questions over MetaGPT.', name='summary_tool_walmart', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [ ]:
system_message = """ \
You are an agent designed to answer queries over a set of given earnings call transcripts reports.
Please always use the tools provided to answer a question. Please use the oringal words in the document as much as possible. Do not rely on prior knowledge.\
"""

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

# Define the agentWorker and agentRunner
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=system_message,
    verbose=True
)

# Initialize the AgentRunner with the agent worker
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "specific use case of AI in their business"
)
print(str(response))

Added user message to memory: specific use case of AI in their business
=== Calling Function ===
Calling function: summary_tool_walmart with args: {"input": "specific use case of AI in their business"}
=== Function Output ===
The specific use case of AI in their business involves leveraging generative AI to enhance the customer experience by improving the product catalog, creating or improving over 850 million pieces of data in the catalog. This use of AI has helped in providing high-quality images of product packaging to associates picking online orders, enabling them to quickly find the items customers are looking for. Additionally, AI-powered search on their app and site, along with a new shopping assistant providing advice and ideas, showcases their commitment to enhancing the customer experience through innovative AI applications.
=== Calling Function ===
Calling function: summary_tool_shop with args: {"input": "specific use case of AI in their business"}
=== Function Output ===
O

In [ ]:
response = agent.chat(
    "Does any of the companies work with Palantir?"
)

Added user message to memory: Does any of the companies work with Palantir?
=== LLM Response ===
Yes, Lowe's is working with Palantir to develop AI solutions.
